# Topic 9: Databases, SQLAlchemy, MongoDB
## Objective
Build a lunch recommendation service and delivery ordering service.

## Set Up
Sign up for [Zomato API](https://developers.zomato.com/api)

Install `sqlalchemy`

In [ ]:
!conda install sqlalchemy!

Install mongodb

### Windows
- https://docs.mongodb.com/manual/installation/


### OS X / linux
Using [Homebrew ](https://brew.sh/)

- `brew tap mongodb/brew`
- `brew install mongodb-community`
- `brew services start mongodb/brew/mongodb-community`

### Pymongo
`!conda install pymongo`

In [163]:
!conda install pymongo -y

In [156]:
from pymongo import MongoClient

In [159]:
client = MongoClient()

## Explore the Zomato documentation

### Warm Up 1
Use the `cities` endpoint to find a city by name. Write a method that does this.

In [74]:
import requests

def get_city_by_name(city_name):
    url = "https://developers.zomato.com/api/v2.1/cities?q={}"
    headers = {"user-key": "7c0538fabc5a6108a754882d2b12b2a9"}
    res = requests.get(url.format(city_name), headers=headers)
    return res.json()['location_suggestions'][0]


In [75]:
get_city_by_name("New York")

{'id': 280,
 'name': 'New York City, NY',
 'country_id': 216,
 'country_name': 'United States',
 'country_flag_url': 'https://b.zmtcdn.com/images/countries/flags/country_216.png',
 'should_experiment_with': 0,
 'has_go_out_tab': 0,
 'discovery_enabled': 1,
 'has_new_ad_format': 0,
 'is_state': 0,
 'state_id': 103,
 'state_name': 'New York State',
 'state_code': 'NY'}

## Create our database

In [77]:
import sqlalchemy
sqlalchemy.__version__

'1.3.20'

In [79]:
from sqlalchemy import create_engine

In [80]:
engine = create_engine('sqlite:///zomato.db', echo = True)

Check if it exists.

In [81]:
conn = engine.connect()

2021-01-22 14:27:11,028 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-01-22 14:27:11,029 INFO sqlalchemy.engine.base.Engine ()
2021-01-22 14:27:11,032 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-01-22 14:27:11,032 INFO sqlalchemy.engine.base.Engine ()


In [83]:
conn.close()

In [84]:
# import os

In [87]:
# os.listdir()

Introspect `engine`

List the table names.

### Exercise
Get all cuisines in a given city.

In [106]:
def get_cuisines_by_city_id(city_id):
    url = "https://developers.zomato.com/api/v2.1/cuisines?city_id={}"
    headers = {"user-key": "7c0538fabc5a6108a754882d2b12b2a9"}
    res = requests.get(url.format(city_id), headers=headers)
    return res.json()['cuisines'][:20]

In [107]:
get_cuisines_by_city_id(280)

[{'cuisine': {'cuisine_id': 1035, 'cuisine_name': 'Afghan'}},
 {'cuisine': {'cuisine_id': 6, 'cuisine_name': 'Afghani'}},
 {'cuisine': {'cuisine_id': 152, 'cuisine_name': 'African'}},
 {'cuisine': {'cuisine_id': 1, 'cuisine_name': 'American'}},
 {'cuisine': {'cuisine_id': 954, 'cuisine_name': 'Amish'}},
 {'cuisine': {'cuisine_id': 151, 'cuisine_name': 'Argentine'}},
 {'cuisine': {'cuisine_id': 175, 'cuisine_name': 'Armenian'}},
 {'cuisine': {'cuisine_id': 3, 'cuisine_name': 'Asian'}},
 {'cuisine': {'cuisine_id': 131, 'cuisine_name': 'Australian'}},
 {'cuisine': {'cuisine_id': 201, 'cuisine_name': 'Austrian'}},
 {'cuisine': {'cuisine_id': 193, 'cuisine_name': 'BBQ'}},
 {'cuisine': {'cuisine_id': 955, 'cuisine_name': 'Bagels'}},
 {'cuisine': {'cuisine_id': 5, 'cuisine_name': 'Bakery'}},
 {'cuisine': {'cuisine_id': 227, 'cuisine_name': 'Bar Food'}},
 {'cuisine': {'cuisine_id': 132, 'cuisine_name': 'Belgian'}},
 {'cuisine': {'cuisine_id': 270, 'cuisine_name': 'Beverages'}},
 {'cuisine': {'

## Defining a model
Let's define the city model.

In [91]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey, Boolean, DateTime


Base = declarative_base()

In [119]:
class City(Base):
    __tablename__ = 'cities'

    city_id = Column(Integer, primary_key=True)
    zomato_id = Column(Integer)
    name = Column(String(100))
    date_created = Column(DateTime, default=datetime.datetime.now)
    
    def __repr__(self):
        return "{}: {}".format(self.zomato_id, self.name)

Check the tables.

In [96]:
engine.table_names()

2021-01-22 14:41:11,859 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-01-22 14:41:11,861 INFO sqlalchemy.engine.base.Engine ()


[]

What s going on?

In [98]:
Base.metadata.create_all(engine)

2021-01-22 14:46:44,468 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("cities")
2021-01-22 14:46:44,471 INFO sqlalchemy.engine.base.Engine ()
2021-01-22 14:46:44,474 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("cities")
2021-01-22 14:46:44,476 INFO sqlalchemy.engine.base.Engine ()
2021-01-22 14:46:44,490 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE cities (
	city_id INTEGER NOT NULL, 
	zomato_id INTEGER, 
	name VARCHAR(100), 
	date_created DATETIME, 
	PRIMARY KEY (city_id)
)


2021-01-22 14:46:44,491 INFO sqlalchemy.engine.base.Engine ()
2021-01-22 14:46:44,502 INFO sqlalchemy.engine.base.Engine COMMIT


### Exercise
Define the cuisine model based on the data available and create it.

- zomato_id
- city_id
- name

In [121]:
class Cuisine(Base):
    __tablename__ = 'cuisines'

    cuisine_id = Column(Integer, primary_key=True)
    zomato_id = Column(Integer)
    name = Column(String(100))
    date_created = Column(DateTime, default=datetime.datetime.now)
    
    def __repr__(self):
        return "{}: {}".format(self.zomato_id, self.name)

In [135]:
Base.metadata.create_all(engine)

2021-01-22 15:17:15,421 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("cities")
2021-01-22 15:17:15,494 INFO sqlalchemy.engine.base.Engine ()
2021-01-22 15:17:15,496 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("cuisines")
2021-01-22 15:17:15,497 INFO sqlalchemy.engine.base.Engine ()
2021-01-22 15:17:15,499 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("cuisines")
2021-01-22 15:17:15,500 INFO sqlalchemy.engine.base.Engine ()
2021-01-22 15:17:15,503 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE cuisines (
	cuisine_id INTEGER NOT NULL, 
	zomato_id INTEGER, 
	name VARCHAR(100), 
	date_created DATETIME, 
	PRIMARY KEY (cuisine_id)
)


2021-01-22 15:17:15,503 INFO sqlalchemy.engine.base.Engine ()
2021-01-22 15:17:15,506 INFO sqlalchemy.engine.base.Engine COMMIT


## Creating instances and saving

In [108]:
from sqlalchemy.orm import sessionmaker

In [109]:
Session = sessionmaker(bind=engine)

In [112]:
session = Session()

In [110]:
new_city = City(name='Some city', zomato_id=99923)

In [113]:
session.add(new_city)

In [116]:
our_city = session.query(City).first()

2021-01-22 14:57:13,818 INFO sqlalchemy.engine.base.Engine SELECT cities.city_id AS cities_city_id, cities.zomato_id AS cities_zomato_id, cities.name AS cities_name, cities.date_created AS cities_date_created 
FROM cities
 LIMIT ? OFFSET ?
2021-01-22 14:57:13,819 INFO sqlalchemy.engine.base.Engine (1, 0)


In [117]:
our_city

In [120]:
our_city.name

'Some city'

### Exercise
Create a script to populate the Cuisine records in the database for Boston, New York and Los Angeles.

In [136]:
city_names = ['Boston', 'New York', 'Los Angeles']
for city in city_names:
    city_id = get_city_by_name(city)['id']
    cuisines = get_cuisines_by_city_id(city_id)
    for cuisine in cuisines:
        cuiz = Cuisine(name=cuisine["cuisine"]["cuisine_name"], zomato_id=cuisine["cuisine"]["cuisine_id"])
        session.add(cuiz)

In [128]:
session.rollback()

In [137]:
all_cuis = session.query(Cuisine).all()

2021-01-22 15:17:25,849 INFO sqlalchemy.engine.base.Engine INSERT INTO cuisines (zomato_id, name, date_created) VALUES (?, ?, ?)
2021-01-22 15:17:25,856 INFO sqlalchemy.engine.base.Engine (1035, 'Afghan', '2021-01-22 15:17:25.849251')
2021-01-22 15:17:25,885 INFO sqlalchemy.engine.base.Engine INSERT INTO cuisines (zomato_id, name, date_created) VALUES (?, ?, ?)
2021-01-22 15:17:25,889 INFO sqlalchemy.engine.base.Engine (152, 'African', '2021-01-22 15:17:25.885774')
2021-01-22 15:17:25,890 INFO sqlalchemy.engine.base.Engine INSERT INTO cuisines (zomato_id, name, date_created) VALUES (?, ?, ?)
2021-01-22 15:17:25,891 INFO sqlalchemy.engine.base.Engine (1, 'American', '2021-01-22 15:17:25.890559')
2021-01-22 15:17:25,892 INFO sqlalchemy.engine.base.Engine INSERT INTO cuisines (zomato_id, name, date_created) VALUES (?, ?, ?)
2021-01-22 15:17:25,896 INFO sqlalchemy.engine.base.Engine (151, 'Argentine', '2021-01-22 15:17:25.892495')
2021-01-22 15:17:25,897 INFO sqlalchemy.engine.base.Engine 

2021-01-22 15:17:25,962 INFO sqlalchemy.engine.base.Engine INSERT INTO cuisines (zomato_id, name, date_created) VALUES (?, ?, ?)
2021-01-22 15:17:25,963 INFO sqlalchemy.engine.base.Engine (270, 'Beverages', '2021-01-22 15:17:25.962751')
2021-01-22 15:17:25,964 INFO sqlalchemy.engine.base.Engine INSERT INTO cuisines (zomato_id, name, date_created) VALUES (?, ?, ?)
2021-01-22 15:17:25,965 INFO sqlalchemy.engine.base.Engine (159, 'Brazilian', '2021-01-22 15:17:25.964503')
2021-01-22 15:17:25,966 INFO sqlalchemy.engine.base.Engine INSERT INTO cuisines (zomato_id, name, date_created) VALUES (?, ?, ?)
2021-01-22 15:17:25,967 INFO sqlalchemy.engine.base.Engine (182, 'Breakfast', '2021-01-22 15:17:25.966280')
2021-01-22 15:17:25,968 INFO sqlalchemy.engine.base.Engine INSERT INTO cuisines (zomato_id, name, date_created) VALUES (?, ?, ?)
2021-01-22 15:17:25,969 INFO sqlalchemy.engine.base.Engine (133, 'British', '2021-01-22 15:17:25.968380')
2021-01-22 15:17:25,970 INFO sqlalchemy.engine.base.En

In [139]:
for city in all_cuis[:5]:
    print(city.name)

Afghan
African
American
Argentine
Armenian


## Querying / Filtering

Let's get order cuisines alphabetically.


In [147]:
for c in session.query(Cuisine).order_by(Cuisine.name)[:10]:
    print(c)

2021-01-22 15:57:19,120 INFO sqlalchemy.engine.base.Engine SELECT cuisines.cuisine_id AS cuisines_cuisine_id, cuisines.zomato_id AS cuisines_zomato_id, cuisines.name AS cuisines_name, cuisines.date_created AS cuisines_date_created 
FROM cuisines ORDER BY cuisines.name
 LIMIT ? OFFSET ?
2021-01-22 15:57:19,121 INFO sqlalchemy.engine.base.Engine (10, 0)
1035: Afghan
1035: Afghan
1035: Afghan
6: Afghani
152: African
152: African
152: African
1: American
1: American
1: American


Let's get all the distict names.

In [169]:
distinct_names = []
for c in session.query(Cuisine).distinct(Cuisine.name):
    distinct_names.append(c.name)

2021-01-22 16:10:27,063 INFO sqlalchemy.engine.base.Engine SELECT DISTINCT cuisines.cuisine_id AS cuisines_cuisine_id, cuisines.zomato_id AS cuisines_zomato_id, cuisines.name AS cuisines_name, cuisines.date_created AS cuisines_date_created 
FROM cuisines
2021-01-22 16:10:27,065 INFO sqlalchemy.engine.base.Engine ()


['Afghan',
 'African',
 'American',
 'Argentine',
 'Armenian',
 'Asian',
 'Australian',
 'BBQ',
 'Bagels',
 'Bakery']

Let's dedupe.

For each name, get all the records that match the name, pick one to be the survivor and delete the others.

2021-01-22 16:13:17,533 INFO sqlalchemy.engine.base.Engine SELECT cuisines.cuisine_id AS cuisines_cuisine_id, cuisines.zomato_id AS cuisines_zomato_id, cuisines.name AS cuisines_name, cuisines.date_created AS cuisines_date_created 
FROM cuisines 
WHERE cuisines.name = ?
2021-01-22 16:13:17,549 INFO sqlalchemy.engine.base.Engine ('Afghan',)


[1035: Afghan, 1035: Afghan, 1035: Afghan]

In [173]:
for name in distinct_names:
    records = list(session.query(Cuisine).filter(Cuisine.name == name))
    survivor = records[0]
    dupes = records[1:]
    for dupe in dupes:
        session.delete(dupe)

2021-01-22 16:15:23,858 INFO sqlalchemy.engine.base.Engine SELECT cuisines.cuisine_id AS cuisines_cuisine_id, cuisines.zomato_id AS cuisines_zomato_id, cuisines.name AS cuisines_name, cuisines.date_created AS cuisines_date_created 
FROM cuisines 
WHERE cuisines.name = ?
2021-01-22 16:15:23,859 INFO sqlalchemy.engine.base.Engine ('Afghan',)
2021-01-22 16:15:23,865 INFO sqlalchemy.engine.base.Engine DELETE FROM cuisines WHERE cuisines.cuisine_id = ?
2021-01-22 16:15:23,867 INFO sqlalchemy.engine.base.Engine ((21,), (41,))
2021-01-22 16:15:23,897 INFO sqlalchemy.engine.base.Engine SELECT cuisines.cuisine_id AS cuisines_cuisine_id, cuisines.zomato_id AS cuisines_zomato_id, cuisines.name AS cuisines_name, cuisines.date_created AS cuisines_date_created 
FROM cuisines 
WHERE cuisines.name = ?
2021-01-22 16:15:23,914 INFO sqlalchemy.engine.base.Engine ('African',)
2021-01-22 16:15:23,918 INFO sqlalchemy.engine.base.Engine DELETE FROM cuisines WHERE cuisines.cuisine_id = ?
2021-01-22 16:15:23,9

2021-01-22 16:15:24,055 INFO sqlalchemy.engine.base.Engine DELETE FROM cuisines WHERE cuisines.cuisine_id = ?
2021-01-22 16:15:24,056 INFO sqlalchemy.engine.base.Engine ((40,), (58,))
2021-01-22 16:15:24,058 INFO sqlalchemy.engine.base.Engine SELECT cuisines.cuisine_id AS cuisines_cuisine_id, cuisines.zomato_id AS cuisines_zomato_id, cuisines.name AS cuisines_name, cuisines.date_created AS cuisines_date_created 
FROM cuisines 
WHERE cuisines.name = ?
2021-01-22 16:15:24,059 INFO sqlalchemy.engine.base.Engine ('Burger',)
2021-01-22 16:15:24,061 INFO sqlalchemy.engine.base.Engine DELETE FROM cuisines WHERE cuisines.cuisine_id = ?
2021-01-22 16:15:24,062 INFO sqlalchemy.engine.base.Engine (59,)
2021-01-22 16:15:24,064 INFO sqlalchemy.engine.base.Engine SELECT cuisines.cuisine_id AS cuisines_cuisine_id, cuisines.zomato_id AS cuisines_zomato_id, cuisines.name AS cuisines_name, cuisines.date_created AS cuisines_date_created 
FROM cuisines 
WHERE cuisines.name = ?
2021-01-22 16:15:24,065 INFO

2021-01-22 16:15:24,137 INFO sqlalchemy.engine.base.Engine ('Breakfast',)
2021-01-22 16:15:24,139 INFO sqlalchemy.engine.base.Engine SELECT cuisines.cuisine_id AS cuisines_cuisine_id, cuisines.zomato_id AS cuisines_zomato_id, cuisines.name AS cuisines_name, cuisines.date_created AS cuisines_date_created 
FROM cuisines 
WHERE cuisines.name = ?
2021-01-22 16:15:24,140 INFO sqlalchemy.engine.base.Engine ('British',)
2021-01-22 16:15:24,142 INFO sqlalchemy.engine.base.Engine SELECT cuisines.cuisine_id AS cuisines_cuisine_id, cuisines.zomato_id AS cuisines_zomato_id, cuisines.name AS cuisines_name, cuisines.date_created AS cuisines_date_created 
FROM cuisines 
WHERE cuisines.name = ?
2021-01-22 16:15:24,143 INFO sqlalchemy.engine.base.Engine ('Bubble Tea',)
2021-01-22 16:15:24,145 INFO sqlalchemy.engine.base.Engine SELECT cuisines.cuisine_id AS cuisines_cuisine_id, cuisines.zomato_id AS cuisines_zomato_id, cuisines.name AS cuisines_name, cuisines.date_created AS cuisines_date_created 
FROM 

Let's find which ones start with "A"

In [1]:
list(session.query(Cuisine).filter(Cuisine.name.in_(['Asian', 'Italian'])))

NameError: name 'session' is not defined

Get the records for Asian, Chinese, and Italian

## Mini-Lab 1: Recommendations & Reviews
Let's build a recommendation engine for users.

### Step 1: Pull in restaurants from Zomato
For Boston and New York.

### Step 2: Table Creation
There are three new tables we need to create

- Restaurant (with the relevant fields)
- User (with username and user_id)
- UserCuisine (to store user's cuisine preferences)
- Recommendations (which links to restaurant and user)

### Step 3: Create_User
Create a helper function that creates a user and also adds UserCuisine preferences

```python
>>> create_user(username='alberto', preferences['Italian', 'Chinese'], city='Boston')
```

### Step 4: Simple matchmaking algo
Pick 5 random restaurants that match their preferences and save to the database.

### (Optional) Step 5: Don't repeat the last 3 recommendations
Need to query the DB in the algo.

### Step 6: Intro to Mongo and reviews